## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the number of neurons
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.constraints import MaxNorm

In [2]:
# Function to create model, required for KerasClassifier
def create_model(neurons):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(4)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # return values
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed = 7)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

x1     x2    x3    x4     x5    x6     x7    x8    Y
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0  1.0
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0  0.0
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0  1.0
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0  0.0
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0  1.0
..    ...    ...   ...   ...    ...   ...    ...   ...  ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0  0.0
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0  0.0
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0  0.0
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0  1.0
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0  0.0

[768 rows x 9 columns]

In [7]:
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

In [8]:
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]

In [9]:
param_grid = dict(model__neurons=neurons)

In [10]:
param_grid

{'model__neurons': [1, 5, 10, 15, 20, 25, 30]}

In [11]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [12]:
grid_result = grid.fit(X, Y)

Epoch 1/100
77/77 [==============================] - 1s 3ms/step - loss: 0.6684 - accuracy: 0.6432
Epoch 2/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6509 - accuracy: 0.6615
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6326 - accuracy: 0.6628
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6332 - accuracy: 0.6615
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6365 - accuracy: 0.6562
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6314 - accuracy: 0.6654
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6261 - accuracy: 0.6732
Epoch 8/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6260 - accuracy: 0.6745
Epoch 9/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6171 - accuracy: 0.6810
Epoch 10/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6200 - accuracy: 0.6836
Epoch 11/

In [13]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.716146 using {'model__neurons': 20}


In [14]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.707031 (0.005524) with: {'model__neurons': 1}
0.713542 (0.017566) with: {'model__neurons': 5}
0.708333 (0.012075) with: {'model__neurons': 10}
0.712240 (0.010253) with: {'model__neurons': 15}
0.716146 (0.018688) with: {'model__neurons': 20}
0.701823 (0.012890) with: {'model__neurons': 25}
0.704427 (0.010253) with: {'model__neurons': 30}
